<a href="https://colab.research.google.com/github/clementsiegrist/segmentation_probabilisticDL/blob/main/pre_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!wget https://raw.githubusercontent.com/clementsiegrist/segmentation_probabilisticDL/main/requirements.txt 
!pip3 install -r requirements.txt
!pip3 install --upgrade --force-reinstall --no-deps kaggle

In [2]:
%%capture
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json


api_token = {"username":"bendidihab","key":"6b534264cbd90d4cc3f040b1894cb226"}
import json
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)


!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!kaggle competitions download -c data-science-bowl-2018
!unzip /content/{/content}/competitions/data-science-bowl-2018/data-science-bowl-2018.zip
!mkdir /content/train
!mv -i /content/stage1_train.zip /content/train
!cd /content/train && unzip /content/train/stage1_train.zip
!mkdir /content/data && mkdir /content/data/train && mkdir /content/data/test
!rm -rf /content/stage1_sample_submission.csv.zip /content/stage1_solution.csv.zip /content/stage1_test.zip /content/stage1_train_labels.csv.zip /content/stage2_sample_submission_final.csv.zip /content/stage2_test_final.zip

In [3]:
import cv2 as cv
import os
import numpy as np
from google.colab.patches import cv2_imshow

In [4]:
i = 1
train_percent = 0.7
data_size = 670
dim = (640,480)
path = '/content/train'
list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]
train_size = int(data_size*train_percent)
test_size = data_size - train_size
for sub in list_subfolders_with_paths :
  folder_types = [f.path for f in os.scandir(sub) if f.is_dir()]
  if i < train_size : 
    os.mkdir('data/train/OP'+str(i))
  else :
    os.mkdir('data/test/OP'+str(i))
  for f_type in folder_types :
    if f_type.split('/')[-1] == 'masks': 
      onlyfiles = [f for f in os.listdir(f_type) if os.path.isfile(os.path.join(f_type, f))]
      src = f_type + '/' + onlyfiles[0]
      src1 = cv.imread(src)
      for img in onlyfiles : 
        src2 = cv.imread(f_type + '/' + img)
        alpha = 1
        beta = 1
        src1 = cv.addWeighted(src1, alpha, src2, beta, 0.0)
      img = cv.resize(src1, dim, interpolation = cv.INTER_AREA)
      if i < train_size :
        cv.imwrite('data/train/OP'+str(i)+'/img_' + str(i) + '_class.png',img)
      else :
        cv.imwrite('data/test/OP'+str(i)+'/img_' + str(i) + '_class.png',img)
    if f_type.split('/')[-1] == 'images':
      onlyfiles = [f for f in os.listdir(f_type) if os.path.isfile(os.path.join(f_type, f))]
      img_raw = cv.imread(f_type + '/' + onlyfiles[0])
      img = cv.resize(img_raw, dim, interpolation = cv.INTER_AREA)
      if i < train_size :
        cv.imwrite('data/train/OP'+str(i)+'/img_' + str(i) + '_raw.png',img)
      else :
        cv.imwrite('data/test/OP'+str(i)+'/img_' + str(i) + '_raw.png',img)
  i += 1
!rm -rf /content/train /content/{

In [5]:
%%capture
!git clone https://github.com/yuta-hi/pytorch-trainer
!cd pytorch-trainer && python setup.py install
!git clone https://github.com/IhabBendidi/pytorch_bayesian_unet 
!cd pytorch_bayesian_unet && pip install -r requirements.txt && python setup.py install


!cp -r /content/data/train /content/pytorch_bayesian_unet/examples/cellular_segmentation/train
!cp -r /content/data/test /content/pytorch_bayesian_unet/examples/cellular_segmentation/test


In [18]:
!python /content/pytorch_bayesian_unet/examples/cellular_segmentation/preprocess.py

# train images: 467
100% 467/467 [00:00<00:00, 2117.91it/s]
# train labels: 467
100% 467/467 [00:02<00:00, 178.77it/s]
# test images: 203
100% 203/203 [00:00<00:00, 1617.91it/s]
# test labels: 203
100% 203/203 [00:01<00:00, 178.83it/s]


In [ ]:
!python /content/pytorch_bayesian_unet/examples/cellular_segmentation/train_and_test_epistemic.py --iteration 2000

GPU: cuda:0
# Minibatch-size: 2

# classes: 2
# samples:
-- train: 421
-- valid: 46
iteration   main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
0                       1.85258e+08                          0.329827                  9.40561       
100         97237       205057                0.950368       0.809718                  346.268       
     total [##................................................]  5.00%
this epoch [#######################...........................] 47.51%
       100 iter, 0 epoch / 2000 iterations
       inf iters/sec. Estimated time to finish: 0:00:00.
200         57748.2     130938                0.941022       0.826521                  683.225       
     total [#####.............................................] 10.00%
this epoch [###############################################...] 95.01%
       200 iter, 0 epoch / 2000 iterations
   0.29678 iters/sec. Estimated time to finish: 1:41:05.036512.
300         32160.9

In [8]:
!python /content/pytorch_bayesian_unet/examples/cellular_segmentation/train_and_test_epistemic.py --test_on_test

GPU: cuda:0
# Minibatch-size: 2

# samples:
-- test: 203
Loaded a snapshot: logs/predictor_iter_00000150.pth
Traceback (most recent call last):
  File "/content/pytorch_bayesian_unet/examples/cellular_segmentation/train_and_test_epistemic.py", line 414, in <module>
    main()
  File "/content/pytorch_bayesian_unet/examples/cellular_segmentation/train_and_test_epistemic.py", line 401, in main
    test_phase(predictor, test, args)
  File "/content/pytorch_bayesian_unet/examples/cellular_segmentation/train_and_test_epistemic.py", line 167, in test_phase
    pred, uncert = infer.run()
  File "/usr/local/lib/python3.6/dist-packages/pytorch_bcnn-1.1.0-py3.6.egg/pytorch_bcnn/inference/inferencer.py", line 193, in run
  File "/usr/local/lib/python3.6/dist-packages/pytorch_bcnn-1.1.0-py3.6.egg/pytorch_bcnn/inference/inferencer.py", line 151, in predict
  File "/usr/local/lib/python3.6/dist-packages/pytorch_bcnn-1.1.0-py3.6.egg/pytorch_bcnn/inference/inferencer.py", line 162, in predict_core
  F

In [9]:
!cd ..

In [10]:
!git clone https://github.com/clementsiegrist/deep_seg.git

fatal: destination path 'deep_seg' already exists and is not an empty directory.


In [11]:
 !pip install keras==2.4 tensorflow==2.3 scikit-learn scikit-image numpy pillow matplotlib imutils pillow opencv-python 

In [12]:
cd /content/deep_seg/

/content/deep_seg


In [13]:
train_path = "/content/drive/MyDrive/Colab Notebooks/cells/train"
validation_path = "/content/drive/MyDrive/Colab Notebooks/cells/validation"
test_path = "/content/drive/MyDrive/Colab Notebooks/cells/test"
test_batch_size = 4
checkpoint_path = '/content/weights'
input_size = (256, 256) 
target_size = (256, 256) # (height, width) of input_size should be divisible by 32
num_class = 2
# fine tuning
train_batch_size = 4
test_batch_size = 4
validation_batch_size = 4
learning_rate = 1e-4
nb_epochs = 10

In [14]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [15]:
from unet_pipeline.main import main
from unet_pipeline.generator import load_pretrained, train_val_gen
import tensorflow.keras as keras

Using TensorFlow backend.


In [16]:
import tensorflow.keras as keras
model_name = 'cell_seg.hdf5'
model = main(train_path, train_batch_size, validation_path, validation_batch_size, nb_epochs)
predict_on_test_and_plot(test_path, model_name=model_name, num=98)

NameError: ignored

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
from PIL import Image
import cv2

model_name = 'cell_seg.hdf5'
predict_on_test_and_plot(test_path, model_name=model_name, num=98)